## 1 Summary, background, value proposition

This data set will be used to train and/or evaluate the performance of a multi-head classifier model for screening sensitive sociopolitical topics, thereby helping curb manipulative online content seeking to influence the political process, such as an election, passage of legislation, and content for campaigning purposes. The data set contains excerpts from social media posts, news articles, and other online content related to various sociopolitical topics. The data set also includes entries labeled as non-sensitive to improve/evaluate the model's precision.

A key question about this data set is whether/by how much senstive and non-sensitive entries overlap, because such overlap may affect the quality of the model and/or the efficacy of the evaluation. Topic modeling techniques were used to answer this question. The results demonstrate that senstive and non-sensitive entries, at least in this data set, do not have significant overlap.

In [1]:
import pathlib
import pandas as pd

data_set = pathlib.Path('./Data_Set_Select_Sociopolitical_Topics.csv')
df = pd.read_csv(data_set)

display(df)


,Sr.No,RawQuery,Sensitive Topic
0,5,No not elected. Slipped in as a runner up to ...,1
1,28,"GREAT WOMAN, GREAT LEGISLATOR, AND A GREAT FRI...",0
2,31,I LOVE what they did with the residential scho...,0
3,37,Wonder what would have happened if everybody h...,0
4,38,And least but not last those undoubtedly 'trut...,1
...,...,...,...
2629,3934,"So basically, a whiny complaining column to su...",0
2630,3936,Oh come off it already. Wearing a beard is no...,0
2631,3941,"Please note that any criticism of Wavemaker, a...",0
2632,3944,Lukashenkos days are numbered. His troops have...,1


## 2 Data acquisition, preprocessing


The data set was scraped from various online socia media sources. I labeled the data points as sensitive or non-sensitive based on the sociopolitical topics discussed in the entries.

Since all the entries are natural language narratives, I performed pre-processing steps such as tokenization, stop word removal, lemmatization, etc.

## 4 Data analysis

I performed topic modeling to answer the question about the overlap between sensitive and non-sensitive entries; to do so, I separated the data set into sensitive and non-sensitive data sets. The results of the topic modeling are shown in the code below. The lists of top words indicate that sensitive and non-sensitive entries have different topics. The intertopic distance maps under the "Data visualization section" also show that sensitive and non-sensitive entries do not have significant overlap. Even for senstive and non-sensitive topics are appear in similar areas on the maps, the term frequency charts show dissimilarities in the terms making up the topics. 

In [2]:
import re
from gensim.utils import tokenize
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation
custom_stopwords = ['displaystyle']

def preprocess_tokenize_with_gensim(text):
  # convert to lowercase, remove extra whitespace
  text = text.lower().strip()
  # remove \n, strip symbols and punctuation 
  text = strip_punctuation(text)
  # remove stopwords
  text = remove_stopwords(text)
  # tokenize
  tokens = list(tokenize(text))
  # remove any additional stopwords if needed (this is a custom extra step if 
  # you see words in the topics that don't belong)
  # remove any words shorter than 2 characters
  tokens = [token for token in tokens if token not in custom_stopwords and len(token) > 2]
  return tokens

# create tokens column
df['paragraph_tokens'] = df['RawQuery'].apply(preprocess_tokenize_with_gensim)
df.head()

,Sr.No,RawQuery,Sensitive Topic,paragraph_tokens
0,5,No not elected. Slipped in as a runner up to ...,1,"[elected, slipped, runner, majority, voted, me..."
1,28,"GREAT WOMAN, GREAT LEGISLATOR, AND A GREAT FRI...",0,"[great, woman, great, legislator, great, frien..."
2,31,I LOVE what they did with the residential scho...,0,"[love, residential, schools]"
3,37,Wonder what would have happened if everybody h...,0,"[wonder, happened, everybody, personal, gun, c..."
4,38,And least but not last those undoubtedly 'trut...,1,"[undoubtedly, truthful, polls]"


In [3]:
# create a dataframe with data from RawQuery column that are sensitive
df_sensitive = df[df['Sensitive Topic'] == 1]

# create a dataframe with data from RawQuery column that are not sensitive
df_not_sensitive = df[df['Sensitive Topic'] == 0]

In [4]:
# Perform Topic Modeling Through Latent Semantic Analysis with term frequency–inverse document frequency (TFIDF)
import os.path
from gensim.corpora import Dictionary
from gensim.models import LsiModel, TfidfModel
import re
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Generate dictionary and document-term-matrix needed for computing TFIDF
dictionary_sensitive = Dictionary(df_sensitive['paragraph_tokens'])
corpus_sensitive = [dictionary_sensitive.doc2bow(text) for text in df_sensitive['paragraph_tokens']]
tfidf_sensitive = TfidfModel(corpus_sensitive)

dictionary_not_sensitive = Dictionary(df_not_sensitive['paragraph_tokens'])
corpus_not_sensitive = [dictionary_not_sensitive.doc2bow(text) for text in df_not_sensitive['paragraph_tokens']]
tfidf_not_sensitive = TfidfModel(corpus_not_sensitive)

In [6]:
# Specific the number of topics
total_sensitive_topics = 10
total_not_sensitive_topics = 10

In [7]:
# Train LDA models using word counts

from gensim.models import LdaModel
lda_sensitive = LdaModel(corpus_sensitive, id2word=dictionary_sensitive, num_topics=total_sensitive_topics)

lda_not_sensitive = LdaModel(corpus_not_sensitive, id2word=dictionary_not_sensitive, num_topics=total_not_sensitive_topics)

In [8]:
# Compare the top 7 words for the top 7 topics from the LDA model
for topic in lda_sensitive.show_topics(num_topics=10, num_words=7, formatted=False):
    print(topic)

for topic in lda_not_sensitive.show_topics(num_topics=10, num_words=7, formatted=False):
    print(topic)
    

(0, [('trump', 0.013975104), ('war', 0.0058105104), ('like', 0.0057645966), ('abortion', 0.0053700036), ('people', 0.004820828), ('government', 0.0042880015), ('president', 0.0042715785)])
(1, [('tax', 0.0055653783), ('like', 0.0053597996), ('ukraine', 0.0052606803), ('abortions', 0.0046332283), ('obama', 0.0039038274), ('abortion', 0.0035686183), ('time', 0.0035468133)])
(2, [('russia', 0.006701567), ('war', 0.0065509537), ('people', 0.005680682), ('oil', 0.004725558), ('biden', 0.004697117), ('election', 0.004519339), ('russian', 0.0043751947)])
(3, [('people', 0.008780022), ('war', 0.006301507), ('ukraine', 0.0062552695), ('biden', 0.0057788272), ('trump', 0.0057657864), ('right', 0.0050477707), ('state', 0.0042518363)])
(4, [('abortion', 0.0074136285), ('people', 0.0054971557), ('russia', 0.004038318), ('government', 0.0038778859), ('war', 0.0038537497), ('biden', 0.0038250147), ('want', 0.0035243644)])
(5, [('ukraine', 0.0071669663), ('war', 0.0060664183), ('trump', 0.005192065), 

## 3 Data visualization

In [9]:
# Visualize the topics in the LDA model
#!pip install pyldavis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()


In [10]:
map_sensitive = pyLDAvis.gensim_models.prepare(lda_sensitive, corpus_sensitive, dictionary_sensitive)
map_sensitive

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.047590 -0.013222       1        1  13.674809
6     -0.051217  0.063957       2        1  12.031499
2     -0.023494  0.066068       3        1  11.683369
3     -0.017208 -0.046093       4        1  11.023928
8      0.113774  0.045670       5        1   9.951824
1      0.039544 -0.058391       6        1   8.974636
5      0.036909  0.002160       7        1   8.880399
0     -0.005674 -0.000711       8        1   8.488095
4     -0.005017 -0.055539       9        1   7.684248
9     -0.040026 -0.003900      10        1   7.607193, topic_info=          Term        Freq       Total Category  logprob  loglift
2343   ukraine  146.000000  146.000000  Default  30.0000  30.0000
1016     trump  136.000000  136.000000  Default  29.0000  29.0000
132   abortion   60.000000   60.000000  Default  28.0000  28.0000
606    percent   14.000000   14.000000  Default  27.0000  27.0000
107      obama   44.000000   44.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
31       going    3.239411   41.884142  Topic10  -6.2305   0.0166
2369   western    3.134895   37.290289  Topic10  -6.2633   0.0999
110     people    3.328349  108.786516  Topic10  -6.2034  -0.9108
1373       pay    2.922874   25.134892  Topic10  -6.3333   0.4244
908      right    2.998104   55.645072  Topic10  -6.3079  -0.3449

[801 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
715       8  0.465316  abortatives
132       1  0.132699     abortion
132       2  0.082937     abortion
132       3  0.033175     abortion
132       4  0.165874     abortion
...     ...       ...          ...
1273      4  0.102838    zelenskyy
1273      6  0.102838    zelenskyy
1273      7  0.154257    zelenskyy
1273      8  0.102838    zelenskyy
1273     10  0.154257    zelenskyy

[1994 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 7, 3, 4, 9, 2, 6, 1, 5, 10])

In [11]:
map_not_sensitive = pyLDAvis.gensim_models.prepare(lda_not_sensitive, corpus_not_sensitive, dictionary_not_sensitive)
map_not_sensitive

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.033029 -0.024743       1        1  13.139462
8     -0.046406  0.028929       2        1  11.491614
1     -0.044519 -0.013584       3        1  11.362677
0     -0.008219 -0.020048       4        1  11.180338
5     -0.013129  0.082144       5        1   9.730767
4      0.004044 -0.043167       6        1   9.395551
6     -0.014855 -0.055995       7        1   8.795620
3      0.076968 -0.044077       8        1   8.762753
2     -0.018899  0.045386       9        1   8.479482
7      0.098044  0.045155      10        1   7.661735, topic_info=        Term        Freq       Total Category  logprob  loglift
352    right   86.000000   86.000000  Default  30.0000  30.0000
171   church   31.000000   31.000000  Default  29.0000  29.0000
367     time  109.000000  109.000000  Default  28.0000  28.0000
259    years   96.000000   96.000000  Default  27.0000  27.0000
541  society   23.000000   23.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
367     time    5.068426  109.432334  Topic10  -6.3001  -0.5033
114     good    4.896025  105.081259  Topic10  -6.3348  -0.4974
365    think    4.893244  106.998979  Topic10  -6.3353  -0.5160
239   people    4.771885  186.323664  Topic10  -6.3604  -1.0958
352    right    4.485306   86.267028  Topic10  -6.4224  -0.3877

[769 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
489       1  0.128923   able
489       2  0.343795   able
489       5  0.128923   able
489       6  0.171897   able
489       7  0.042974   able
...     ...       ...    ...
1053      6  0.188061    yes
1053      7  0.107463    yes
1053      8  0.053732    yes
1053      9  0.134329    yes
1007     10  0.905230  yucca

[2146 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 9, 2, 1, 6, 5, 7, 4, 3, 8])